</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization (Refactoring ...)

In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

endpoint_id = "vllm-vbzs7iyoydoidq"
api_key = os.environ["RUNPOD_API_KEY"]

get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key) # Unlimited LLM endpoints

Could not load vllm class, check CUDA support and GPU RAM size


In [3]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=20, num_runs=2) # Scale up !

No code block found in the response.
No code block found in the response.
Best code has structural fitness 0.33, functional fitness 1.00, global fitness 0.67

📊 Code 0: Fitness: 8.3%
--------------------------------------------------------------------------------
❌ Error Messages:
0


📊 Code 1: Fitness: 66.7%


📊 Code 2: Fitness: 0.0%


📊 Code 3: Fitness: 8.3%
--------------------------------------------------------------------------------
❌ Error Messages:
0


📊 Code 4: Fitness: 66.7%
--------------------------------------------------------------------------------
❌ Error Messages:
0


📊 Code 5: Fitness: 66.7%


📊 Code 6: Fitness: 0.0%


📊 Code 7: Fitness: 0.0%


📊 Code 8: Fitness: 0.0%


📊 Code 9: Fitness: 0.0%


📊 Code 10: Fitness: 33.3%


📊 Code 11: Fitness: 66.7%


📊 Code 12: Fitness: 8.3%
--------------------------------------------------------------------------------
❌ Error Messages:
0


📊 Code 13: Fitness: 8.3%
------------------------------------------------------------------

</div>
<div align="center">
  <img src="img/Project-Nirvana-evolve.gif" width="500" alt="Fourier reconstruction convergence">
  <p><em> Evolve a population of nodes. </em></p>
</div>

In [5]:
# Population building phase ... 
from methods.llm import get_groq_response, get_claude_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=3, meta_prompt=mp, get_response=get_endpoint_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

Best code has structural fitness 0.33, functional fitness 0.50, global fitness 0.42
Based on the provided information, I'll analyze the effectiveness of the current evolution strategy:

1. Fitness Improvement:
- The current best fitness is 1.0 (from the initial population)
- The offspring solution has a fitness of approximately 0.417
- This indicates that the evolution strategy has not improved fitness, as the offspring's fitness is lower than the parent solutions

2. Implementation Improvements:

The offspring solution shows several implementation improvements compared to the parents:

Positive Changes:
1. Added actual Google search functionality through Serper API
2. Introduced error handling with try/except blocks
3. Added type hints for better code readability and maintainability
4. Included more structured API interaction with proper HTTP connection handling
5. Implemented JSON parsing and formatting of search results

Limitations/Issues:
1. The solution is more complex and potent


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="900" alt="Fourier reconstruction convergence">
  <p><em> Plan, and evolve the plans. </em></p>
</div>


In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

endpoint_id = "vllm-vbzs7iyoydoidq"
api_key = os.environ["RUNPOD_API_KEY"]

get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key) # Unlimited LLM endpoints

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan
from methods.evolnode import PlanNode


# Initialize PlanNode 
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])
plan = PlanNode(mp, get_endpoint_response)

# i1 evolution of plan
plan_dicts, err_msg = plan.evolve_plan_dict(method="i1", batch_size=20) # Batch_size of 100 gives no slow-down

for plan_dict in plan_dicts:
    visualize_plan_dict(plan_dict, plan.meta_prompt.task)
    break

# Manual input on main-node test cases 
main_test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

is_sucess, err_msg = plan.spawn_test_cases(main_test_cases) #  pinned test cases generation

 :: Evolving 20 plans in parallel...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 :: Pseudo-code generated for each plan
 :: Plan_dict generated for each plan


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 188.9805ms
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitl

Spawned 2 test cases for all sub-nodes


In [5]:
plan.evolve_sub_nodes() # sub-node evolution | what's taking so long?

# Bug above
# ---> 86 is_aligned, error_msg_delta = metric(pred_value, target_value)
#      87 if not is_aligned:
#      88     error_msg += error_msg_delta + "\n"

# TypeError: 'tuple' object is not callable

KeyboardInterrupt: 